In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from cvxopt import matrix,solvers,blas
import cvxopt as opt
import pickle

In [ ]:
#市值平方根加权回归计算每日的因子收益率
def factor_return_calcu(alpha_df,factor_list):
    grouped=alpha_df.groupby("datetime")
    date_list=list(alpha_df["datetime"].drop_duplicates())
    code_list=list(alpha_df["code"].drop_duplicates())
    factor_names=factor_list


    factor_return=pd.DataFrame(index=date_list[:-21],columns=range(len(factor_list)+28))

    i = 0
    while(1):
        print(i)
        if i+21>=len(date_list):
            break

        date=date_list[i]
        df=pd.DataFrame(grouped.get_group(date))
        sqrt_mktv_array=np.sqrt(np.array(df["market_value"]))

        #市值平方根加权OLS回归
        return_value = np.array(df["21d_ret"]) * sqrt_mktv_array
        
        #风格因子
        x=np.empty(shape=(len(sqrt_mktv_array),0))
        for factor_name in factor_names:
            factor_value = np.array(df[factor_name])*sqrt_mktv_array
            x=np.column_stack([x,factor_value])
        
        #行业哑变量
        dummy_var = np.array(sm.categorical(np.array(df["industry_code"])))[:, 1:]
        for j in range(dummy_var.shape[1]):
            dummy_var[:,j]=dummy_var[:,j]*sqrt_mktv_array
        x=np.column_stack([x,dummy_var])

        model = sm.OLS(return_value, x)
        results = model.fit()

        factor_return.loc[date,:]=results.params

        i = i+1

    factor_return.to_csv("factor_return.csv",index_label="date")

alpha_df=pd.read_csv("equal_weight ZScore.csv").fillna(0)
factor_list=['BETA','RSTR','LNCAP','ETOP','DASTD','EGRO','BTOP','DTOA','STOM']
factor_return_calcu(alpha_df,factor_list)


In [ ]:
#市值平方根加权回归计算每日股票特质性收益
def spe_return_calcu(alpha_df,factor_list):
    grouped = alpha_df.groupby("datetime")
    date_list = list(alpha_df["datetime"].drop_duplicates())
    code_list = list(alpha_df["code"].drop_duplicates())
    factor_names = factor_list

    spe_return=pd.DataFrame(index=code_list,columns=date_list[:-21])

    i = 0
    while(1):
        print(i)
        if i+21>=len(date_list):
            break

        date=date_list[i]
        df=pd.DataFrame(grouped.get_group(date))
        sqrt_mktv_array=np.sqrt(np.array(df["market_value"]))

        #市值平方根加权OLS回归
        return_value = np.array(df["21d_ret"]) * sqrt_mktv_array
        
        #风格因子
        x = np.empty(shape=(len(sqrt_mktv_array), 0))
        for factor_name in factor_names:
            factor_value = np.array(df[factor_name])*sqrt_mktv_array
            x=np.column_stack([x,factor_value])
        
        #行业哑变量
        dummy_var = np.array(sm.categorical(np.array(df["industry_code"])))[:, 1:]
        for j in range(dummy_var.shape[1]):
            dummy_var[:,j]=dummy_var[:,j]*sqrt_mktv_array
        x=np.column_stack([x,dummy_var])

        model = sm.OLS(return_value, x)
        results = model.fit()

        resid=(return_value-results.fittedvalues)/sqrt_mktv_array
        dic=dict(zip(np.array(df["code"]),resid))
        spe_return[date]=code_list
        spe_return[date]=spe_return[date].apply(lambda x:dic.get(x,0))

        i = i+1

    spe_return.to_csv("spe_return.csv",index_label="code")

alpha_df=pd.read_csv("equal_weight ZScoreequal_weight ZScore.csv").fillna(0)
factor_list=['BETA','RSTR','LNCAP','ETOP','DASTD','EGRO','BTOP','DTOA','STOM']
spe_return_calcu(alpha_df,factor_list)




In [ ]:
#计算因子收益的协方差矩阵（原本还进行了New-West调整，但是发现调整之后风险会出现持续高估的情况，最后就把New-West调整舍去了）
NW_dict={}

def New_West(date_sect,dt):

    k=date_sect
    date_list = list(dt.index.drop_duplicates())
    date=date_list[date_sect]

    print(date)
    
    #计算各时期的权重（时间越近，权重越高）
    lamd=pow(0.5,1/90)
    weight=np.power(lamd,np.arange(252))
    weight=weight/np.sum(weight)
    
    
    #计算因子收益的协方差矩阵
    factor_num=len(dt.columns)
    dt=np.array(dt)
    def f1(index):
        i=int(index/factor_num)
        j=int(index%factor_num)
        f_1 = dt[k + 21:k + 273, i]
        f_2 = dt[k + 21:k + 273, j]
        return np.sum(weight*(f_1-f_1.mean())*(f_2-f_2.mean()))#+2/3*np.sum(weight[:-1] * (f_1[1:] - f_1.mean()) * (f_2[:-1] - f_2.mean()))\
               #+2/3*np.sum(weight[:-1] * (f_1[:-1] - f_1.mean()) * (f_2[1:] - f_2.mean()))+1/3*np.sum(weight[:-2] * (f_1[2:] - f_1.mean()) * (f_2[:-2] - f_2.mean()))\
               #+1/3*np.sum(weight[:-2] * (f_1[:-2] - f_1.mean()) * (f_2[2:] - f_2.mean()))
    pf1=np.vectorize(f1,otypes=[float])
    F_NW=pf1(np.arange(factor_num**2)).reshape(factor_num,factor_num)
    
    #将因子收益协方差矩阵放入字典中
    NW_dict[date]=F_NW
    print("finish")

    return 0

data=pd.read_csv("factor_return.csv",engine="python",encoding="utf-8")
data.set_index("date",inplace=True)
for i in range(len(data.index)-294):
    New_West(i,data)

print(NW_dict)
f=open("New_West.txt","wb")
pickle.dump(NW_dict,f,0)
f.close()


In [ ]:
#计算股票特质性收益的波动率（同样是原本还进行了New-West调整，但是发现调整之后风险会出现持续高估的情况，最后就把New-West调整舍去了）
NW_dict={}

def spec_NW(date_sect,data):
    print(date_sect)
    k = date_sect
    date_list = list(data.columns.drop_duplicates())
    date = date_list[date_sect]
    
    #计算各个日期的权重（时间越近，权重越高）
    lamd=pow(0.5,1/90)
    weight=np.power(lamd,np.arange(252))
    weight=weight/np.sum(weight)
    
    #计算股票特质性收益的方差
    stock_num=len(data.index)
    data = np.array(data)
    def f1(i):
        f_1 = data[i, k + 21:k + 273]
        return np.sum(weight * (f_1 - f_1.mean()) ** 2)
    pf1=np.vectorize(f1,otypes=[float])
    F_raw=pf1(np.arange(stock_num))

    def f2(i):
        f_1 = data[i, k + 21:k + 273]
        return np.sum(weight[:-1] * (f_1[:-1] - f_1.mean()) * (f_1[1:] - f_1.mean()))
    pf2=np.vectorize(f2,otypes=[float])
    C_lag_1=pf2(np.arange(stock_num))

    def f3(i):
        f_1 = data[i, k + 21:k + 273]
        return np.sum(weight[:-2] * (f_1[:-2] - f_1.mean()) * (f_1[2:] - f_1.mean()))
    pf3=np.vectorize(f3,otypes=[float])
    C_lag_2=pf3(np.arange(stock_num))

    F_NW=F_raw#+4/3*C_lag_1+2/3*C_lag_2
    
    #将股票特质性收益的方差存入字典中
    NW_dict[date]=F_NW

    return F_NW

data=pd.read_csv("spe_return.csv",index_col="code").fillna(0)
for i in range(len(data.columns)-295):
    spec_NW(i,data)

print(NW_dict)

f=open("spec_NW.txt","wb")
pickle.dump(NW_dict,f,0)
f.close()




In [ ]:
'''
#对因子收益的协方差矩阵进行特征值调整,策略中未使用
Eig_dict={}

def Eigen_adjust(cov_matrix):
    factor_num=cov_matrix.shape[0]
    eigvalue,eigvector=np.linalg.eig(cov_matrix)

    bias_vec=np.zeros(factor_num,dtype=float)
    
    #进行1000次模拟，算出平均偏差
    for j in range(1000):
        eigfactor_matrix=np.empty(shape=(0,252),dtype=float)
        for i in range(factor_num):
            vec=np.random.normal(0,np.sqrt(eigvalue[i]),252)
            eigfactor_matrix=np.row_stack([eigfactor_matrix,vec])

        factor_matrix=np.dot(eigvector,eigfactor_matrix)

        F_MC=np.cov(factor_matrix)

        eigvalue_mc,eigvector_mc=np.linalg.eig(F_MC)

        eigvalue_true=np.diag(np.dot(np.dot(eigvector_mc.transpose(),cov_matrix),eigvector_mc))

        bias=eigvalue_true/eigvalue_mc

        bias_vec+=bias/1000

    #对平均偏差进行旋转
    bias_vec=np.sqrt(bias_vec)
    bias_vec=1.2*(bias_vec-1)+1
    bias_vec=np.power(bias_vec,2)
    eigvalue_adjust=bias_vec*eigvalue
    
    #计算出新的协方差矩阵
    cov_matrix=np.dot(np.dot(eigvector,np.diag(eigvalue_adjust)),eigvector.transpose())

    return cov_matrix

f=open("New_West.txt","rb")
NW_dict=pickle.load(f)
date_list=list(NW_dict.keys())

for date in date_list:
    print(date)
    F_NW=NW_dict[date]
    F_Eigen =Eigen_adjust(F_NW)
    Eig_dict[date]=F_Eigen

f = open("Eigen.txt", "wb")
pickle.dump(Eig_dict, f, 0)
f.close()
'''


In [ ]:
'''
#对股票特质性收益进行贝叶斯压缩，压缩目标为所在市值分组平均风险,策略中未使用
BS_dict={}

def Bayes_shrinkage(specrisk_df):
    #按市值排名进行分组
    mkt_arr=np.array(specrisk_df["market_value"])
    mkt_rank = mkt_arr.argsort().argsort()
    specrisk_df["mkt_rank"]=mkt_rank
    mkt_tag = pd.cut(specrisk_df.mkt_rank, 10, labels=False)
    specrisk_df["tag"] = mkt_tag
    
    #计算出各组风险的平均值和标准差
    avg = specrisk_df[["risk"]].groupby(specrisk_df["tag"]).agg([np.mean])
    std = specrisk_df[["risk"]].groupby(specrisk_df["tag"]).agg([np.std])
    specrisk_df["avg"]=specrisk_df["tag"].apply(lambda x:avg.iloc[x,0])
    specrisk_df["std"] = specrisk_df["tag"].apply(lambda x: std.iloc[x, 0])
    
    #进行贝叶斯压缩估计（系数q设为1）
    q=1
    specrisk_df["v"]=np.abs(q*specrisk_df["risk"]-specrisk_df["avg"])/(np.abs(q*(specrisk_df["risk"]-specrisk_df["avg"]))+specrisk_df["std"])
    specrisk_df["BS_risk"]=specrisk_df["v"]*specrisk_df["avg"]+(1-specrisk_df["v"])*specrisk_df["risk"]
    return list(specrisk_df["BS_risk"])

f=open("spec_NW.txt","rb")
NW_dict=pickle.load(f)
date_list=list(NW_dict.keys())

alpha_df=pd.read_csv("equal_weight ZScore.csv")
grouped=alpha_df.groupby("datetime")

for date in date_list:
    print(date)
    F_NW=NW_dict[date]
    mktv=pd.DataFrame(grouped.get_group(date))["market_value"]
    d={"risk":np.sqrt(F_NW),"market_value":mktv}
    specrisk_df=pd.DataFrame(data=d)
    BS_risk=Bayes_shrinkage(specrisk_df)
    BS_dict[date]=np.power(BS_risk,2)

f=open("spec_BS.txt","wb")
pickle.dump(BS_dict,f,0)
f.close()
'''




In [ ]:
'''
#对因子收益协方差矩阵进行偏误调整,策略中未使用
Bias_dict={}

f=open("Eigen.txt","rb")
Eigen_dict=pickle.load(f)
date_list=list(Eigen_dict.keys())
date_list.sort(reverse=True)

def Bias_adjust(alpha_df,date_sect):
    k = date_sect
    date = date_list[date_sect]
    
    #计算各个日期的权重（时间越近，权重越高）
    lamd = pow(0.5, 1 / 45)
    weight = np.power(lamd, np.arange(126))
    weight = weight / np.sum(weight)
    
    
    f_vol_multi=0
    for i in range(1,127):
        #获得每日风险预测值
        _date_=date_list[k+i]
        risk=np.sqrt(np.diag(Eigen_dict[_date_]))
        
        #计算每日的Bias统计量
        Bias_t=0
        for j in range(len(risk)):
            risk_fore=risk[j]
            Bias_t+=(alpha_df.iloc[i+k+20,j]/risk_fore)**2
        Bias_t/=len(risk)
        
        f_vol_multi+=Bias_t*weight[i-1]

    print(f_vol_multi)
    Bias_dict[date]=f_vol_multi*Eigen_dict[date]



data=pd.read_csv("factor_return.csv",engine="python",encoding="utf-8")
data.set_index("date",inplace=True)
for i in range(len(data.index)-427):
    print(i)
    Bias_adjust(data,i)

f=open("Biasadjust.txt","wb")
pickle.dump(Bias_dict,f,0)
f.close()
'''



In [ ]:
'''
#对股票特质性风险进行偏误调整,策略中未使用
Biasadjust_dict = {}

f = open("spec_BS.txt", "rb")
BS_dict = pickle.load(f)
date_list = list(BS_dict.keys())
date_list.sort(reverse=True)


def spec_Bias_adjust(alpha_df, date_sect):
    k = date_sect
    date = date_list[date_sect]

    print(date)
    
    #计算各个日期的权重（时间越近，权重越高）
    lamd = pow(0.5, 1 / 45)
    weight = np.power(lamd, np.arange(126))
    weight = weight / np.sum(weight)

    spe_vol_multi = 0
    for i in range(1, 127):
        #获得每日的风险预测值
        _date_ = date_list[k + i]
        risk = np.sqrt(BS_dict[_date_])
    
        #计算每日的Bias统计量
        stock_num = len(alpha_df.index)
        stock_range = range(stock_num)
        ret = alpha_df.iloc[stock_range, i+k + 20]
        Bias_t = np.sum(np.power(ret / risk, 2)) / len(risk)
        
        spe_vol_multi += Bias_t * weight[i - 1]

    print(spe_vol_multi)
    Biasadjust_dict[date] = spe_vol_multi * BS_dict[date]


data = pd.read_csv("spe_return.csv", index_col='code').fillna(0)
for i in range(len(data.columns) - 427):
    spec_Bias_adjust(data, i)

f = open("spec_Biasadjust.txt", "wb")
pickle.dump(Biasadjust_dict, f, 0)
f.close()
'''

In [ ]:
#获得基准权重
f=open("index_w_dict.txt","rb")
index_w_dict=pickle.load(f)

weights_dict={}#存放最优化组合权重

alpha_df=pd.read_csv("equal_weight ZScore.csv").fillna(0)

grouped = alpha_df.groupby("datetime")
date_list = list(alpha_df["datetime"].drop_duplicates())
code_list = list(alpha_df["code"].drop_duplicates())

def optimize(date):
    print(date)
    df = pd.DataFrame(grouped.get_group(date))
    n = len(df["code"])
    
    #获得当日基准组合的权重
    index_weight=index_w_dict[date]
    df["weight"]=df["code"].apply(lambda x:index_weight.get(x.replace("SH","XSHG"),0) if "SH" in x\
                                            else index_weight.get(x.replace("SZ","XSHE"),0))
    df["weight"]=df["weight"]*1/sum(df["weight"])

    factor_list =['BETA','LNCAP','RSTR','ETOP','DASTD','EGRO','BTOP','DTOA','STOM']
    X = np.empty(shape=[len(df["code"]), 0])
    for factor_name in factor_list:
        factor_data = np.array(df[factor_name])
        X = np.column_stack([X, factor_data])
    X = X.transpose()#风格因子暴露矩阵


    indus_var = np.array(sm.categorical(np.array(df["industry_code"])))[:, 1:]
    indus_var = np.array(indus_var).transpose()#行业因子暴露矩阵

    mtx=np.row_stack([X,indus_var])#所有股票在风格因子和行业因子上的暴露矩阵

    f=open("New_West.txt","rb")
    factor_cov=pickle.load(f)[date]
    cov_matrix=np.dot(np.dot(mtx.transpose(),factor_cov),mtx)#结构化风险协方差矩阵
    f=open("spec_NW.txt","rb")
    spe_risk=pickle.load(f)[date]#每个股票的特质性风险
    cov_matrix+=np.diag(spe_risk)
    
    #最小化的函数
    P = matrix(cov_matrix)
    q = -matrix(df["avg_ret"])

    #约束权重和为1
    A = matrix(1.0, (1, n))
    b = matrix(1.0)


    I = matrix(0.0, (n, n))
    I[::n + 1] = 1.0
    G_1 = matrix([-I, I])#约束权重上下限
    factor = opt.matrix(mtx[[0,2],:])
    G_2=matrix([-factor,factor])#约束风格因子和行业因子暴露值上下限
    G=matrix([G_1,G_2])
    h_1 = matrix([opt.matrix(n*[0]),opt.matrix(df["weight"]+0.015)])#约束权重上下限
    bench = factor * matrix(df["weight"])
    h_2_high=opt.matrix([x*1.2 if x>0 else x*0.8 for x in bench])
    h_2_low=opt.matrix([x*(-0.8) if x >0 else x*(-1.2) for x in bench])
    h_2=matrix([h_2_low,h_2_high])#约束风格因子和行业因子暴露值上下限
    h=matrix([h_1,h_2])

    dims = {"l": G.size[0], "q": [], "s": []}
    portfolio = solvers.coneqp(P, q, G, h, dims,A,b)["x"]#最优化计算
    returns = blas.dot(q, portfolio)#收益率
    risks = np.sqrt(blas.dot(portfolio, P * portfolio))#结构化风险
    portfolio=np.array(portfolio).flatten()
    
    #打印出每日持仓组合以及收益风险情况
    print(portfolio[portfolio>0.00001])
    print(returns," ",risks)

    port_dict=dict(zip(df["code"],portfolio))
    weights_dict[date]=port_dict

    return 0

f=open(r"day.txt","rb")
dt_list=pickle.load(f)
for date in dt_list[3:]:
    optimize(date)

f=open("weights_dict_3.txt","wb")
pickle.dump(weights_dict,f,0)
f.close()
